In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
import pickle
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from xgboost import XGBRegressor
import seaborn as sns
import math
from sklearn.model_selection import cross_val_score
from scipy import stats
import Utils
from sklearn.preprocessing import PolynomialFeatures


In [ ]:
def split_data(data):
    y_data=data.iloc[:,-1]
    X_data=data.iloc[:,:-1]
    X_train,X_test,y_train,y_test=train_test_split(X_data,y_data,random_state=100,test_size=0.3)
    return X_train,X_test,y_train,y_test

In [ ]:
def get_evaluation_index(y_true,y_predict):
    mae_score=mean_absolute_error(y_true,y_predict)
    rmse_score=math.sqrt(mean_squared_error(y_true,y_predict))
    r2_score_1=r2_score(y_true,y_predict)
    #print("mae_score:%s\nrmse_score:%s\nr2_score:%s"%(mae_score,rmse_score,r2_score_1))
    return mae_score,rmse_score,r2_score_1

In [ ]:
#绘制学习曲线
def plot_learning_curve(algo,num,X_train,X_test,y_train,y_test):
    if len(X_train.shape)==1:
        X_train=[[x] for x in X_train]
        X_test=[[x] for x in X_test]
    train_score=[]
    test_score=[]
    for i in range(1,len(X_train)+1):
        if i%10==0:
            algo.fit(X_train[:i],y_train[:i])
            y_train_predict = algo.predict(X_train[:i])
            train_score.append(mean_absolute_error(y_train[:i],y_train_predict))

            y_test_predict=algo.predict(X_test)
            test_score.append(mean_absolute_error(y_test,y_test_predict))
    plt.xlabel("Train num")
    plt.ylabel("mae score")
    plt.title("feature number:%s"%num)
    plt.plot([i for i in range(1,len(train_score)+1)],np.sqrt(train_score),label="train")
    plt.plot([i for i in range(1,len(train_score)+1)],np.sqrt(test_score),label="test")
    plt.legend()
    #指定X轴和y轴的长度，X轴从0到len(X_train)+1,y轴从0到4
    plt.axis([0,len(train_score)+1,0,1])
    plt.show()

In [ ]:
def get_train_test_data(all_data):
    train_data=all_data[all_data['type']=='train'].drop(['type'],axis=1)
    test_data=all_data[all_data['type']=='test'].drop(['type','target'],axis=1)
    return train_data,test_data

In [ ]:
def split_train_validate_data(train_data,test_size,random_state):
    X=train_data.iloc[:,:-1]
    y=train_data['target']
    X_train,X_validate,y_train,y_validate=train_test_split(X,y,test_size=test_size,random_state=random_state)
    return X_train,X_validate,y_train,y_validate

In [ ]:
#读取数据
data_path='./data/zhengqi_train.txt'
o_data=pd.read_csv(data_path,sep='\t')
#读取测试数据
test_path='./data/zhengqi_test.txt'
test_data=pd.read_csv(test_path,sep='\t')

### 查看测试数据集和训练集的数据分布是否相同，去除掉不同分布的数据特征

In [ ]:
#合并训练集与测试集
o_data['type']='train'
test_data['type']='test'
all_data=pd.concat([o_data,test_data])
#绘制训练集与测试集的分布图
for feature in all_data.columns[:-2]:
    all_data[all_data['type']=='train'][feature].plot(kind='kde')
    all_data[all_data['type']=='test'][feature].plot(kind='kde')
    plt.title(feature)
    plt.legend(['train','test'])
    plt.show()


In [ ]:
#删除掉分布不一样的特征数据 
#通过观察上述分布图发现v11,v13,v14,v17,v19,v2,v21,v35,v22,v27,v5,v9
#删除掉分布不一样的特征，然后对比模型的准确度
#自己挑的：'V11','V13','V14','V17','V19','V2','V21','V35','V22','V27','V5','V9' 提交结果：0.1350
#论坛中的："V5","V9","V11","V17","V22","V28" 提交结果：0.1361
drop_feature_list=['V11','V13','V14','V17','V19','V2','V21','V35','V22','V27','V5','V9']
add_drop_feature=[]
for drop_feature in drop_feature_list:
    add_drop_feature.append(drop_feature)
    all_data_drop=all_data.drop(labels=add_drop_feature,axis=1)
    train_data_drop=all_data_drop[all_data_drop['type']=='train'].drop(labels=['type'],axis=1)
    X_train,X_test,y_train,y_test=split_data(train_data_drop)
    xgb_reg=XGBRegressor(max_depth=8,min_child_weight=5,eta=0.05, gamma=0.025,colsample_bytree= 0.6,subsample=0.7)
    xgb_reg.fit(X_train,y_train)
    y_predict=xgb_reg.predict(X_test)
    mae_score,rmse_score,r2_score_1=get_evaluation_index(y_test,y_predict)
    plot_learning_curve(xgb_reg,len(X_train.columns),X_train,X_test,y_train,y_test)
    print("drop feature:%s\n mae:%s\t rmse:%s\t r2:%s\n"%([feature for feature in add_drop_feature],mae_score,rmse_score,r2_score_1))

In [ ]:
#保存drop_feature之后的结果
all_data_drop.to_csv('./re/all_data_drop.txt',header=True,index=False)

In [ ]:
all_data_drop_1=pd.read_csv('./re/all_data_drop.txt')

In [ ]:
test_data_drop=all_data_drop[all_data_drop['type']=='test'].drop(['type','target'],axis=1)
y_predict=xgb_reg.predict(test_data_drop)
y_predict_df=pd.DataFrame(y_predict)
y_predict_df.to_csv('./re/y_predict_1.txt',header=False,index=False)

### 通过皮尔逊相关系数 去掉相关性弱的特征

In [ ]:
train_data=all_data_drop_1[all_data_drop_1['type']=='train'].drop(['type'],axis=1)

In [ ]:
# 找出相关程度
plt.figure(figsize=(20, 16))  # 指定绘图对象宽度和高度
colnm = train_data.columns.tolist()  # 列表头
pearson_corr = train_data[colnm].corr(method="pearson")  # 相关系数矩阵，即给出了任意两个变量之间的相关系数
mask = np.zeros_like(pearson_corr, dtype=np.bool)  # 构造与mcorr同维数矩阵 为bool型
mask[np.triu_indices_from(mask)] = True  # 角分线右侧为True
cmap = sns.diverging_palette(220, 10, as_cmap=True)  # 返回matplotlib colormap对象

g = sns.heatmap(pearson_corr, mask=mask, cmap=cmap, square=True, annot=True, fmt='0.2f')  # 热力图（看两两相似度）
plt.show()

In [ ]:
# 找出相关程度
plt.figure(figsize=(20, 16))  # 指定绘图对象宽度和高度
colnm = train_data.columns.tolist()  # 列表头
spearman_corr = train_data[colnm].corr(method="spearman")  # 相关系数矩阵，即给出了任意两个变量之间的相关系数
mask = np.zeros_like(spearman_corr, dtype=np.bool)  # 构造与mcorr同维数矩阵 为bool型
mask[np.triu_indices_from(mask)] = True  # 角分线右侧为True
cmap = sns.diverging_palette(220, 10, as_cmap=True)  # 返回matplotlib colormap对象
g = sns.heatmap(spearman_corr, mask=mask, cmap=cmap, square=True, annot=True, fmt='0.2f')  # 热力图（看两两相似度）
plt.show()

In [ ]:
s1=spearman_corr['target']
s2=pearson_corr['target']
com_between_corr=pd.merge(left=s1,right=s2,on=s1.index).set_index('key_0').rename(columns={'target_x':'spearman_corr','target_y':'pearson_corr'})
com_between_corr

In [ ]:
### 分别根据pearson 和spearman 相关系数去除无用特征
#首先设置阈值为0.1，去除相关性小于0.1的特征
thresholding=0.1
drop_feature_by_spearman=com_between_corr[np.abs(com_between_corr['spearman_corr'])<=thresholding].index
drop_feature_by_pearson=com_between_corr[np.abs(com_between_corr['pearson_corr'])<=thresholding].index
print("spearman:%s"%drop_feature_by_pearson)
print("pearson:%s"%drop_feature_by_spearman)

In [ ]:
def get_drop_feature_by_corr(data,thresholding=0.1,corr_name='spearman'):
    train_data=data[data['type']=='train'].drop(['type'],axis=1)
    corr = train_data[colnm].corr(method=corr_name)  
    s1=corr['target']
    drop_feature=s1[np.abs(s1)<=thresholding].index
    return drop_feature

In [ ]:
def get_train_data_drop_corr(data,drop_feature):
    all_data_drop_feature_by_corr=data.drop(labels=drop_feature,axis=1)
    train_data_drop=all_data_drop_feature_by_corr[all_data_drop_feature_by_corr['type']=='train'].drop(labels=['type'],axis=1)
    test_data_drop=all_data_drop_feature_by_corr[all_data_drop_feature_by_corr['type']=='test'].drop(labels=['type','target'],axis=1)
    return all_data_drop_feature_by_corr,train_data_drop,test_data_drop

In [ ]:
def result_to_save(re,path):
    y_predict_df=pd.DataFrame(y_predict)
    y_predict_df.to_csv(path,header=False,index=False)

In [ ]:
para_dict=[]
for corr_name in ['spearman','pearson']:
    for thresholding in [0.1,0.15,0.2,0.3,0.4,0.6]:
        #设定相关性及阈值获取需要删除的特征
        drop_feature=get_drop_feature_by_corr(all_data_drop_1,thresholding,corr_name)
        train_data_drop,test_data_drop=get_train_data_drop_corr(all_data_drop_1)
        xgb_reg=XGBRegressor(n_estimators=1000,max_depth=8,min_child_weight=5,eta=0.05, gamma=0.025,colsample_bytree= 0.6,subsample=0.7)
        data=train_data_drop
        y_data=data.iloc[:,-1]
        X_data=data.iloc[:,:-1]
        scores=cross_val_score(xgb_reg,X_data,y_data,scoring='neg_mean_absolute_error',cv=10)
        para_dict.append((corr_name,thresholding,scores))
        print("corr_name:%s\tthresholding:%f\tMAE:%s"%(corr_name,thresholding,np.mean(-scores)))

In [ ]:
drop_feature=get_drop_feature_by_corr(all_data_drop_1,0.2,'spearman')
all_data_drop_feature_by_corr,train_data_drop,X_test=get_train_data_drop_corr(all_data_drop_1,drop_feature)
all_data_drop_feature_by_corr.to_csv('./re/all_data_drop_by_corr.txt',header=True,index=False)
xgb_reg=XGBRegressor(max_depth=8,min_child_weight=5,eta=0.05, gamma=0.025,colsample_bytree= 0.6,subsample=0.7)
y_data=train_data_drop.iloc[:,-1]
X_data=train_data_drop.iloc[:,:-1]
xgb_reg.fit(X_data,y_data)
y_predict=xgb_reg.predict(X_test)
result_to_save(y_predict,'./re/y_predict_1.txt')

### 对数据进行标准化

In [ ]:
all_data_drop_by_corr=pd.read_csv('./re/all_data_drop_by_corr.txt')

In [ ]:
all_data_drop_by_corr.columns

In [ ]:
def min_max_normal(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))

In [ ]:
normal_col=all_data_drop_by_corr.columns[:-2]
all_data_drop_by_corr[normal_col]=all_data_drop_by_corr[normal_col].apply(min_max_normal,axis=0)

In [ ]:
#保存归一化后的数据
all_data_drop_by_corr.to_csv('./re/all_data_normal.txt',index=False,header=True)

In [ ]:
#读取归一化后的数据并进行预测
all_data_normal=pd.read_csv('./re/all_data_normal.txt')
train_data,test_data=get_train_test_data(all_data_normal)
X_train,X_validate,y_train,y_validate=split_train_validate_data(train_data,test_size=0.3,random_state=1)

In [ ]:
xgb_reg=XGBRegressor(max_depth=6,min_child_weight=5,eta=0.05, gamma=0.025,colsample_bytree= 0.6,subsample=0.7)
X_data=train_data.iloc[:,:-1]
y_data=train_data['target']
scores=cross_val_score(xgb_reg,X_data,y_data,cv=10,scoring='neg_mean_absolute_error')
print(-np.mean(scores))

In [ ]:
xgb_reg.fit(X_data,y_data)
y_predict=xgb_reg.predict(test_data)
result_to_save(y_predict,'./re/y_predict_1.txt')

成绩从0.1357提高至0.1341

In [ ]:
df=pd.read_csv('./re/all_data_normal.txt')

In [ ]:
df.describe()

### BOX-COX transform

In [ ]:
all_data_normal=pd.read_csv('./re/all_data_normal.txt')
cols_numeric=all_data_normal.columns

In [ ]:
fcols = 6
frows = len(cols_numeric)-1
plt.figure(figsize=(4*fcols,4*frows))
i=0

for var in cols_numeric:
    if var!='target' and var !='type':
        dat = all_data_normal[[var, 'target']].dropna()
        
        i+=1
        plt.subplot(frows,fcols,i)
        sns.distplot(dat[var],fit=stats.norm);
        plt.title(var+' Original')
        plt.xlabel('')
        
        
        i+=1
        plt.subplot(frows,fcols,i)
        _=stats.probplot(dat[var], plot=plt)
        plt.title('skew='+'{:.4f}'.format(stats.skew(dat[var])))
        plt.xlabel('')
        plt.ylabel('')
        
        i+=1
        plt.subplot(frows,fcols,i)
        plt.plot(dat[var], dat['target'],'.',alpha=0.5)
        plt.title('corr='+'{:.2f}'.format(np.corrcoef(dat[var], dat['target'])[0][1]))
        
       
        """
        boxcox:
           y = (x**lmbda - 1) / lmbda,  for lmbda > 0
               log(x),                  for lmbda = 0
        """
     
        i+=1
        plt.subplot(frows,fcols,i)
        trans_var, lambda_var = stats.boxcox(dat[var].dropna()+1)
        trans_var = min_max_normal(trans_var)      
        sns.distplot(trans_var , fit=stats.norm);
        plt.title(var+' Tramsformed')
        plt.xlabel('')
        
        i+=1
        plt.subplot(frows,fcols,i)
        _=stats.probplot(trans_var, plot=plt)
        plt.title('skew='+'{:.4f}'.format(stats.skew(trans_var)))
        plt.xlabel('')
        plt.ylabel('')
        
        i+=1
        plt.subplot(frows,fcols,i)
        plt.plot(trans_var, dat['target'],'.',alpha=0.5)
        plt.title('corr='+'{:.2f}'.format(np.corrcoef(trans_var,dat['target'])[0][1]))

In [ ]:
#进行box-cox变换
for feature in cols_numeric[:-2]:
    all_data_normal[feature],_=stats.boxcox(all_data_normal[feature]+1)

In [ ]:
#保存box-cox变换后的结果
all_data_normal.to_csv("./re/all_data_box_cox.txt",index=False,header=True)

In [ ]:
all_data_box_cox=pd.read_csv('./re/all_data_box_cox.txt')
train_data,test_data=get_train_test_data(all_data_box_cox)
X_train,X_validate,y_train,y_validate=split_train_validate_data(train_data,test_size=0.3,random_state=1)

In [ ]:
xgb_reg=XGBRegressor(max_depth=6,min_child_weight=5,eta=0.05, gamma=0.025,colsample_bytree= 0.6,subsample=0.7)
X_data=train_data.iloc[:,:-1]
y_data=train_data['target']
scores=cross_val_score(xgb_reg,X_data,y_data,cv=10,scoring='neg_mean_absolute_error')
print(-np.mean(scores))

In [ ]:
xgb_reg.fit(X_data,y_data)
y_predict=xgb_reg.predict(test_data)
result_to_save(y_predict,'./re/y_predict_1.txt')

结论 对特征做box-cox变换后 分数从0.1341降为0.1342

### 对相关度高的特征进行多项式化

In [ ]:
all_data_box_cox=pd.read_csv('./re/all_data_box_cox.txt')
feature_thresholding=0.75
all_data_box_cox_train=all_data_box_cox[all_data_box_cox['type']=='train'].drop(['type'],axis=1)
target_corr=all_data_box_cox_train.corr('spearman')['target']
high_relation_features=target_corr[target_corr>feature_thresholding].index[:-1]
print(high_relation_features)

In [ ]:
poly=PolynomialFeatures(degree=2,interaction_only=False)
poly_features=poly.fit_transform(all_data_box_cox_train[high_relation_features])

In [ ]:
poly_df=pd.DataFrame(data=poly_features,columns=poly.get_feature_names(high_relation_features)).iloc[:,1:]
train_poly_df=poly_df.drop(labels=high_relation_features,axis=1)

In [ ]:
train_poly_df

In [ ]:
all_data_box_cox_test=all_data_box_cox[all_data_box_cox['type']=='test'].drop(['type','target'],axis=1)

In [ ]:
poly_features_test=poly.transform(all_data_box_cox_test[high_relation_features])
poly_df_test=pd.DataFrame(data=poly_features_test,columns=poly.get_feature_names(high_relation_features)).iloc[:,1:]
poly_df_test=poly_df_test.drop(labels=high_relation_features,axis=1)
all_poly_df=train_poly_df.append(poly_df_test,ignore_index=True)

In [ ]:
target_type=all_data_box_cox[['target','type']]
all_data_box_cox=all_data_box_cox.drop(labels=['target','type'],axis=1)

In [ ]:
all_data_box_cox_poly=pd.merge(all_data_box_cox,all_poly_df,left_index=True,right_index=True,how='outer')
all_data_box_cox_poly=pd.merge(all_data_box_cox_poly,target_type,left_index=True,right_index=True,how='outer')

In [ ]:
all_data_box_cox_poly.to_csv('./re/all_data_box_cox_poly.txt',header=True,index=False)

### 对多项式特征，通过xgb训练筛选出离群数据

In [ ]:
from sklearn.metrics import make_scorer

def rmse(y_true, y_pred):
    diff = y_pred - y_true
    sum_sq = sum(diff**2)    
    n = len(y_pred)   
    
    return np.sqrt(sum_sq/n)
def mse(y_ture,y_pred):
    return mean_squared_error(y_ture,y_pred)
mse_score=make_scorer(mse,greater_is_better=False)

In [ ]:
def find_outliers(model, X, y, sigma=3):

    # predict y values using model
    try:
        y_pred = pd.Series(model.predict(X), index=y.index)
    # if predicting fails, try fitting the model first
    except:
        model.fit(X,y)
        y_pred = pd.Series(model.predict(X), index=y.index)
        
    # calculate residuals between the model prediction and true y values
    resid = y - y_pred
    mean_resid = resid.mean()
    std_resid = resid.std()

    # calculate z statistic, define outliers to be where |z|>sigma
    z = (resid - mean_resid)/std_resid    
    outliers = z[abs(z)>sigma].index
    
    # print and plot the results
    print('R2=',model.score(X,y))
    print('rmse=',rmse(y, y_pred))
    print("mse=",mean_squared_error(y,y_pred))
    print('---------------------------------------')

    print('mean of residuals:',mean_resid)
    print('std of residuals:',std_resid)
    print('---------------------------------------')

    print(len(outliers),'outliers:')
    print(outliers.tolist())

    plt.figure(figsize=(15,5))
    ax_131 = plt.subplot(1,3,1)
    plt.plot(y,y_pred,'.')
    plt.plot(y.loc[outliers],y_pred.loc[outliers],'ro')
    plt.legend(['Accepted','Outlier'])
    plt.xlabel('y')
    plt.ylabel('y_pred');

    ax_132=plt.subplot(1,3,2)
    plt.plot(y,y-y_pred,'.')
    plt.plot(y.loc[outliers],y.loc[outliers]-y_pred.loc[outliers],'ro')
    plt.legend(['Accepted','Outlier'])
    plt.xlabel('y')
    plt.ylabel('y - y_pred');

    ax_133=plt.subplot(1,3,3)
    z.plot.hist(bins=50,ax=ax_133)
    z.loc[outliers].plot.hist(color='r',bins=50,ax=ax_133)
    plt.legend(['Accepted','Outlier'])
    plt.xlabel('z')
    
    plt.savefig('outliers.png')
    
    return outliers

'./re/all_data_box_cox.txt'


In [ ]:
all_data_box_cox=pd.read_csv('./re/all_data_box_cox_poly.txt')
train_data,test_data=get_train_test_data(all_data_box_cox)
X_train,X_validate,y_train,y_validate=split_train_validate_data(train_data,test_size=0.3,random_state=1)
xgb_reg=XGBRegressor(max_depth=6,min_child_weight=5,eta=0.05, gamma=0.025,colsample_bytree= 0.6,subsample=0.7)
outliers = find_outliers(xgb_reg,train_data.iloc[:,:-1],train_data['target'])

In [ ]:
all_data_drop_out_data=all_data_box_cox.drop(labels=outliers)
all_data_drop_out_data.to_csv('./re/all_data_drop_out_data.txt',header=True,index=False)

In [ ]:
train_data,test_data=get_train_test_data(all_data_drop_out_data)
X_train,X_validate,y_train,y_validate=split_train_validate_data(train_data,test_size=0.3,random_state=1)

In [ ]:
from sklearn import metrics

In [ ]:
xgb_reg=XGBRegressor(max_depth=6,min_child_weight=5,eta=0.05, gamma=0.025,colsample_bytree= 0.6,subsample=0.7)
X_data=train_data.iloc[:,:-1]
y_data=train_data['target']
scores=cross_val_score(xgb_reg,X_data,y_data,cv=10,scoring='neg_mean_squared_error')
print(-np.mean(scores))

In [ ]:
xgb_reg.fit(X_data,y_data)
y_predict=xgb_reg.predict(test_data)
result_to_save(y_predict,'./re/y_predict_1.txt')

总结：通过xgb筛选出离群数据，score从0.1342提高到0.1336

### 对xgb进行调参

In [ ]:
all_data_drop_out_data=pd.read_csv('./re/all_data_drop_out_data.txt')
train_data,test_data=get_train_test_data(all_data_drop_out_data)
X_data=train_data.iloc[:,:-1]
y_data=train_data['target']

In [ ]:
xgb_reg=XGBRegressor(booster='gbtree',max_depth=10,
                     min_child_weight=6,
                     gamma=0.025,colsample_bytree= 0.9,
                     subsample=0.1,reg_lambda=0.5,
                     reg_alpha=0,learning_rate=0.009,
                     n_estimators=1000)
grid_param={"max_depth":[9],
           "min_child_weight":[6],
           "subsample":[0.1],
           "colsample_bytree":[0.9],
           "n_estimators":[1000],
           "learning_rate":[0.006,0.007,0.008],
           "reg_lambda":[0.4,0.5,0.6],
           "reg_alpha":[0,0.1,0.2],
           }
grid_model=GridSearchCV(xgb_reg,grid_param,scoring=mse_score,cv=5)
grid_model.fit(X_data,y_data)

In [ ]:
print(grid_model.best_params_)
print(grid_model.best_score_)
print(grid_model.best_estimator_)

In [ ]:
xgb_reg=XGBRegressor(booster='gbtree',max_depth=9,
                     min_child_weight=6,
                     gamma=0.025,colsample_bytree= 0.9,
                     subsample=0.1,reg_lambda=0.5,
                     reg_alpha=0,learning_rate=0.008,
                     n_estimators=1000)
X_data=train_data.iloc[:,:-1]
y_data=train_data['target']
scores=cross_val_score(xgb_reg,X_data,y_data,cv=10,scoring='neg_mean_squared_error')
print(-np.mean(scores))

0.120

In [ ]:
xgb_reg.fit(X_data,y_data)
y_predict=xgb_reg.predict(test_data)
result_to_save(y_predict,'./re/y_predict_1.txt')

结论：经过对xgb调参score从0.1336提高到0.1251